## Date Handling of Input Files

> The data contains periods in CET/CEST format.
	

* we need to have start and end date in one consitent time zone
* we need to split the period into start and end date of the period 

"01/01/2015 00:00:00 - 01/01/2015 00:15:00"


In [1]:
# "31/03/2024 01:15:00 - 31/03/2024 01:30:00"			
# "31/03/2024 01:30:00 - 31/03/2024 01:45:00 (CET)"		
# "31/03/2024 01:45:00 (CET) - 31/03/2024 03:00:00 (CEST)"	
# "31/03/2024 03:00:00 (CEST) - 31/03/2024 03:15:00"		
# "31/03/2024 03:15:00 - 31/03/2024 03:30:00"			

# "27/10/2024 01:45:00 - 27/10/2024 02:00:00 (CEST)"		
# "27/10/2024 02:00:00 (CEST) - 27/10/2024 02:15:00 (CEST)"	
# "27/10/2024 02:15:00 (CEST) - 27/10/2024 02:30:00 (CEST)"	
# "27/10/2024 02:30:00 (CEST) - 27/10/2024 02:45:00 (CEST)"	
# "27/10/2024 02:45:00 (CEST) - 27/10/2024 02:00:00 (CET)"	
# "27/10/2024 02:00:00 (CET) - 27/10/2024 02:15:00 (CET)"		
# "27/10/2024 02:15:00 (CET) - 27/10/2024 02:30:00 (CET)"		
# "27/10/2024 02:30:00 (CET) - 27/10/2024 02:45:00 (CET)"		
# "27/10/2024 02:45:00 (CET) - 27/10/2024 03:00:00"		
# "27/10/2024 03:00:00 - 27/10/2024 03:15:00"			


# "26/03/2016 23:00:00 - 27/03/2016 00:00:00"			
# "27/03/2016 00:00:00 - 27/03/2016 01:00:00 (CET)"		
# "27/03/2016 01:00:00 (CET) - 27/03/2016 03:00:00 (CEST)"	
# "27/03/2016 03:00:00 (CEST) - 27/03/2016 04:00:00"		
# "27/03/2016 04:00:00 - 27/03/2016 05:00:00"			

# "30/10/2016 00:00:00 - 30/10/2016 01:00:00"			
# "30/10/2016 01:00:00 - 30/10/2016 02:00:00 (CEST)"		
# "30/10/2016 02:00:00 (CEST) - 30/10/2016 02:00:00 (CET)"	
# "30/10/2016 02:00:00 (CET) - 30/10/2016 03:00:00"		
# "30/10/2016 03:00:00 - 30/10/2016 04:00:00"		

## Setup

All the manipulations and plots in this notebook can be created with standard libraries such as matplotlib, statsmodels etc. 

In [2]:
# Main data packages. 
import numpy as np
import pandas as pd

# Data Viz. 
import statsmodels.formula.api as smf
from statsmodels.tsa.seasonal import seasonal_decompose
from scipy.ndimage import gaussian_filter
from calendar import monthrange
from calendar import month_name

import matplotlib.pyplot as plt
import matplotlib.patches as patches
import seaborn as sns
sns.set_style(
    style='darkgrid', 
    rc={'axes.facecolor': 'white', 'grid.color': '.8'}
)
NF_ORANGE = '#ff5a36'
NF_BLUE = '#163251'
cmaps_hex = ['#193251','#FF5A36','#696969', '#7589A2','#FF5A36', '#DB6668']
sns.set_palette(palette=cmaps_hex)
sns_c = sns.color_palette(palette=cmaps_hex)
%matplotlib inline
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

plt.rcParams['figure.figsize'] = [12, 6]
plt.rcParams['figure.dpi'] = 100

In [3]:
sns_c

[(0.09803921568627451, 0.19607843137254902, 0.3176470588235294),
 (1.0, 0.35294117647058826, 0.21176470588235294),
 (0.4117647058823529, 0.4117647058823529, 0.4117647058823529),
 (0.4588235294117647, 0.5372549019607843, 0.6352941176470588),
 (1.0, 0.35294117647058826, 0.21176470588235294),
 (0.8588235294117647, 0.4, 0.40784313725490196)]

## Import Data 

The data for this notebook was downloaded from the [meteoblue website](https://www.meteoblue.com/en/weather/archive/export/basel_switzerland_2661604) and consits of weather data for the city of Basel from 2008 till 2020. 

In [4]:
files = [
    "../../data/res/off_wind/GUI_WIND_SOLAR_GENERATION_FORECAST_OFFSHORE_201412312300-201512312300.csv",
    "../../data/res/off_wind/GUI_WIND_SOLAR_GENERATION_FORECAST_OFFSHORE_201512312300-201612312300.csv",
    "../../data/res/off_wind/GUI_WIND_SOLAR_GENERATION_FORECAST_OFFSHORE_201612312300-201712312300.csv",
    "../../data/res/off_wind/GUI_WIND_SOLAR_GENERATION_FORECAST_OFFSHORE_201712312300-201812312300 - 1.csv",
    "../../data/res/off_wind/GUI_WIND_SOLAR_GENERATION_FORECAST_OFFSHORE_201712312300-201812312300 - 2.csv",
    "../../data/res/off_wind/GUI_WIND_SOLAR_GENERATION_FORECAST_OFFSHORE_201812312300-201912312300.csv",
    "../../data/res/off_wind/GUI_WIND_SOLAR_GENERATION_FORECAST_OFFSHORE_201912312300-202012312300.csv",
    "../../data/res/off_wind/GUI_WIND_SOLAR_GENERATION_FORECAST_OFFSHORE_202012312300-202112312300.csv",
    "../../data/res/off_wind/GUI_WIND_SOLAR_GENERATION_FORECAST_OFFSHORE_202112312300-202212312300.csv",
    "../../data/res/off_wind/GUI_WIND_SOLAR_GENERATION_FORECAST_OFFSHORE_202212312300-202312312300.csv",
    "../../data/res/off_wind/GUI_WIND_SOLAR_GENERATION_FORECAST_OFFSHORE_202312312300-202412312300.csv",
    "../../data/res/off_wind/GUI_WIND_SOLAR_GENERATION_FORECAST_OFFSHORE_202412312300-202512312300.csv",
    # "../../data/res/off_wind/GUI_WIND_SOLAR_GENERATION_FORECAST_OFFSHORE_202512312300-202612312300.csv",
]

raw_df = pd.concat((pd.read_csv(f, delimiter=",") for f in files), ignore_index=True)


In [5]:
raw_df.shape  #(541275, 6)

(420768, 6)

In [6]:
raw_df.columns

Index(['MTU (CET/CEST)', 'Area', 'Day-ahead (MW)', 'Intraday (MW)',
       'Current (MW)', 'Actual (MW)'],
      dtype='object')

In [7]:
# Create working copy of dataframe
df_utc_op = raw_df.copy()
df_utc_op = df_utc_op[["MTU (CET/CEST)","Day-ahead (MW)", "Actual (MW)"]].rename(  #"Intraday (MW)","Current (MW)",
    columns={
        'MTU (CET/CEST)': 'period',  
        "Day-ahead (MW)": "off_wind_da",
        # "Intraday (MW)": "OffSh Intraday (MW)",
        # "Current (MW)": "OffSh Current (MW)",
        "Actual (MW)": "off_wind_act",
    }
)
df_utc_op.head()

,period,off_wind_da,off_wind_act
0,01/01/2015 00:00:00 - 01/01/2015 00:15:00,258.0,524.39
1,01/01/2015 00:15:00 - 01/01/2015 00:30:00,258.0,521.4
2,01/01/2015 00:30:00 - 01/01/2015 00:45:00,259.0,518.56
3,01/01/2015 00:45:00 - 01/01/2015 01:00:00,259.0,520.36
4,01/01/2015 01:00:00 - 01/01/2015 01:15:00,259.0,520.32


In [8]:
df_utc_op.shape

(420768, 3)

In [9]:
# Split "period" into start/end strings
df_utc_op[["period_start", "period_end"]] = df_utc_op["period"].str.split(" - ", n=1, expand=True)
df_utc_op.head()

,period,off_wind_da,off_wind_act,period_start,period_end
0,01/01/2015 00:00:00 - 01/01/2015 00:15:00,258.0,524.39,01/01/2015 00:00:00,01/01/2015 00:15:00
1,01/01/2015 00:15:00 - 01/01/2015 00:30:00,258.0,521.4,01/01/2015 00:15:00,01/01/2015 00:30:00
2,01/01/2015 00:30:00 - 01/01/2015 00:45:00,259.0,518.56,01/01/2015 00:30:00,01/01/2015 00:45:00
3,01/01/2015 00:45:00 - 01/01/2015 01:00:00,259.0,520.36,01/01/2015 00:45:00,01/01/2015 01:00:00
4,01/01/2015 01:00:00 - 01/01/2015 01:15:00,259.0,520.32,01/01/2015 01:00:00,01/01/2015 01:15:00


In [10]:

def add_timezone_and_utc(df, col): #="period_start"):
    # State: start in CET
    tz_state = "CET"
    tz_list = []

    # Walk rows in order and flip state if original string contains CET/CEST
    for val in df[col].astype(str):
        if "CEST" in val:
            tz_state = "CEST"
        elif "CET" in val:
            tz_state = "CET"
        tz_list.append(tz_state)

    # Parse datetime from the column (remove any existing timezone text)
    base_dt = pd.to_datetime(
        df[col].astype(str).str.replace(r"\s*\(.*\)$", "", regex=True),
        dayfirst=True,
        errors="coerce"
    )

    # Add labeled version
    # df["period_start_labeled"] = (
    df[col + "_labeled"] = (
        base_dt.dt.strftime("%d/%m/%Y %H:%M:%S") + " (" + pd.Series(tz_list, index=df.index) + ")"
    )

    # Compute UTC by subtracting 1 or 2 hours depending on CET/CEST
    offset_hours = pd.Series(tz_list, index=df.index).map({"CET": 1, "CEST": 2})
    df[col + "_utc"] = base_dt - pd.to_timedelta(offset_hours, unit="h")
    # df["tsp_start_utc"] = base_dt - pd.to_timedelta(offset_hours, unit="h")

    return df


In [11]:
df_utc_op = add_timezone_and_utc(df_utc_op, col="period_start")
df_utc_op = add_timezone_and_utc(df_utc_op, col="period_end")

In [12]:
df_utc_op

,period,off_wind_da,off_wind_act,period_start,period_end,period_start_labeled,period_start_utc,period_end_labeled,period_end_utc
0,01/01/2015 00:00:00 - 01/01/2015 00:15:00,258.0,524.39,01/01/2015 00:00:00,01/01/2015 00:15:00,01/01/2015 00:00:00 (CET),2014-12-31 23:00:00,01/01/2015 00:15:00 (CET),2014-12-31 23:15:00
1,01/01/2015 00:15:00 - 01/01/2015 00:30:00,258.0,521.4,01/01/2015 00:15:00,01/01/2015 00:30:00,01/01/2015 00:15:00 (CET),2014-12-31 23:15:00,01/01/2015 00:30:00 (CET),2014-12-31 23:30:00
2,01/01/2015 00:30:00 - 01/01/2015 00:45:00,259.0,518.56,01/01/2015 00:30:00,01/01/2015 00:45:00,01/01/2015 00:30:00 (CET),2014-12-31 23:30:00,01/01/2015 00:45:00 (CET),2014-12-31 23:45:00
3,01/01/2015 00:45:00 - 01/01/2015 01:00:00,259.0,520.36,01/01/2015 00:45:00,01/01/2015 01:00:00,01/01/2015 00:45:00 (CET),2014-12-31 23:45:00,01/01/2015 01:00:00 (CET),2015-01-01 00:00:00
4,01/01/2015 01:00:00 - 01/01/2015 01:15:00,259.0,520.32,01/01/2015 01:00:00,01/01/2015 01:15:00,01/01/2015 01:00:00 (CET),2015-01-01 00:00:00,01/01/2015 01:15:00 (CET),2015-01-01 00:15:00
...,...,...,...,...,...,...,...,...,...
420763,31/12/2025 22:45:00 - 31/12/2025 23:00:00,5425.65,3371.77,31/12/2025 22:45:00,31/12/2025 23:00:00,31/12/2025 22:45:00 (CET),2025-12-31 21:45:00,31/12/2025 23:00:00 (CET),2025-12-31 22:00:00
420764,31/12/2025 23:00:00 - 31/12/2025 23:15:00,5435.78,2618.79,31/12/2025 23:00:00,31/12/2025 23:15:00,31/12/2025 23:00:00 (CET),2025-12-31 22:00:00,31/12/2025 23:15:00 (CET),2025-12-31 22:15:00
420765,31/12/2025 23:15:00 - 31/12/2025 23:30:00,5464.14,2653.33,31/12/2025 23:15:00,31/12/2025 23:30:00,31/12/2025 23:15:00 (CET),2025-12-31 22:15:00,31/12/2025 23:30:00 (CET),2025-12-31 22:30:00
420766,31/12/2025 23:30:00 - 31/12/2025 23:45:00,5497.52,2771.47,31/12/2025 23:30:00,31/12/2025 23:45:00,31/12/2025 23:30:00 (CET),2025-12-31 22:30:00,31/12/2025 23:45:00 (CET),2025-12-31 22:45:00


In [13]:
df_utc_op.columns

Index(['period', 'off_wind_da', 'off_wind_act', 'period_start', 'period_end',
       'period_start_labeled', 'period_start_utc', 'period_end_labeled',
       'period_end_utc'],
      dtype='object')

In [14]:
# Create working copy of dataframe
df_utc_op_temp = df_utc_op.copy()
# select only those values
# df_price = df_price[df_price["Sequence"] != "Sequence Sequence 2"]
 # drop columns
df_utc_op.drop(columns=['period_start', 'period_end', 'period_start_labeled', 'period_end_labeled'], inplace=True)

df_utc_op = df_utc_op.assign(
    date1=lambda x: x["period_start_utc"].dt.date,
    date=lambda x: pd.to_datetime(
        x["date1"],
        format="mixed",
        dayfirst=True,
        errors="coerce"
    ),
    year=lambda x: x["period_start_utc"].dt.year,
    month=lambda x: x["period_start_utc"].dt.month,
    #monthname=lambda x: x["period_start_utc"].dt.month_name(),
    day=lambda x: x["period_start_utc"].dt.day,
    dayofyear=lambda x: x["period_start_utc"].dt.dayofyear,
    hour=lambda x: x["period_start_utc"].dt.hour,
    #week=lambda x: x["period_start_utc"].dt.isocalendar().week.astype(int),
    week=lambda x: x["period_start_utc"].dt.isocalendar().week.astype("Int64"),
    dayofweek=lambda x: x["period_start_utc"].dt.dayofweek,   # 0=Mon ... 6=Sun
    #dayname=lambda x: x["period_start_utc"].dt.day_name(),   

)
df_utc_op.drop(columns=['date1'], inplace=True)

df_utc_op.head()

,period,off_wind_da,off_wind_act,period_start_utc,period_end_utc,date,year,month,day,dayofyear,hour,week,dayofweek
0,01/01/2015 00:00:00 - 01/01/2015 00:15:00,258.0,524.39,2014-12-31 23:00:00,2014-12-31 23:15:00,2014-12-31,2014,12,31,365,23,1,2
1,01/01/2015 00:15:00 - 01/01/2015 00:30:00,258.0,521.4,2014-12-31 23:15:00,2014-12-31 23:30:00,2014-12-31,2014,12,31,365,23,1,2
2,01/01/2015 00:30:00 - 01/01/2015 00:45:00,259.0,518.56,2014-12-31 23:30:00,2014-12-31 23:45:00,2014-12-31,2014,12,31,365,23,1,2
3,01/01/2015 00:45:00 - 01/01/2015 01:00:00,259.0,520.36,2014-12-31 23:45:00,2015-01-01 00:00:00,2014-12-31,2014,12,31,365,23,1,2
4,01/01/2015 01:00:00 - 01/01/2015 01:15:00,259.0,520.32,2015-01-01 00:00:00,2015-01-01 00:15:00,2015-01-01,2015,1,1,1,0,1,3


In [15]:
df_utc_op.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 420768 entries, 0 to 420767
Data columns (total 13 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   period            420768 non-null  object        
 1   off_wind_da       420672 non-null  object        
 2   off_wind_act      420768 non-null  object        
 3   period_start_utc  420768 non-null  datetime64[ns]
 4   period_end_utc    420768 non-null  datetime64[ns]
 5   date              420768 non-null  datetime64[ns]
 6   year              420768 non-null  int32         
 7   month             420768 non-null  int32         
 8   day               420768 non-null  int32         
 9   dayofyear         420768 non-null  int32         
 10  hour              420768 non-null  int32         
 11  week              420768 non-null  Int64         
 12  dayofweek         420768 non-null  int32         
dtypes: Int64(1), datetime64[ns](3), int32(6), object(3)
memory 

In [16]:
# Check NaN only in one column ('price')
rows_with_missing_price = df_utc_op[df_utc_op["off_wind_da"].isna()]
rows_with_missing_price.tail(100)

,period,off_wind_da,off_wind_act,period_start_utc,period_end_utc,date,year,month,day,dayofyear,hour,week,dayofweek
288,04/01/2015 00:00:00 - 04/01/2015 00:15:00,NaN,518.58,2015-01-03 23:00:00,2015-01-03 23:15:00,2015-01-03,2015,1,3,3,23,1,5
289,04/01/2015 00:15:00 - 04/01/2015 00:30:00,NaN,518.95,2015-01-03 23:15:00,2015-01-03 23:30:00,2015-01-03,2015,1,3,3,23,1,5
290,04/01/2015 00:30:00 - 04/01/2015 00:45:00,NaN,519.27,2015-01-03 23:30:00,2015-01-03 23:45:00,2015-01-03,2015,1,3,3,23,1,5
291,04/01/2015 00:45:00 - 04/01/2015 01:00:00,NaN,518.06,2015-01-03 23:45:00,2015-01-04 00:00:00,2015-01-03,2015,1,3,3,23,1,5
292,04/01/2015 01:00:00 - 04/01/2015 01:15:00,NaN,518.62,2015-01-04 00:00:00,2015-01-04 00:15:00,2015-01-04,2015,1,4,4,0,1,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...
379,04/01/2015 22:45:00 - 04/01/2015 23:00:00,NaN,475.55,2015-01-04 21:45:00,2015-01-04 22:00:00,2015-01-04,2015,1,4,4,21,1,6
380,04/01/2015 23:00:00 - 04/01/2015 23:15:00,NaN,471.91,2015-01-04 22:00:00,2015-01-04 22:15:00,2015-01-04,2015,1,4,4,22,1,6
381,04/01/2015 23:15:00 - 04/01/2015 23:30:00,NaN,466.63,2015-01-04 22:15:00,2015-01-04 22:30:00,2015-01-04,2015,1,4,4,22,1,6
382,04/01/2015 23:30:00 - 04/01/2015 23:45:00,NaN,450.64,2015-01-04 22:30:00,2015-01-04 22:45:00,2015-01-04,2015,1,4,4,22,1,6


In [17]:
# Check NaN only in one column ('price')
rows_with_missing_price = df_utc_op[df_utc_op["off_wind_act"].isna()]
rows_with_missing_price

,period,off_wind_da,off_wind_act,period_start_utc,period_end_utc,date,year,month,day,dayofyear,hour,week,dayofweek


In [18]:
# Convert to numeric, turn non-numeric strings into NaN
df_utc_op["off_wind_act"] = pd.to_numeric(df_utc_op["off_wind_act"], errors="coerce")
df_utc_op["off_wind_da"] = pd.to_numeric(df_utc_op["off_wind_da"], errors="coerce")

rows_with_missing_price = df_utc_op[df_utc_op["off_wind_act"].isna()]
rows_with_missing_price

,period,off_wind_da,off_wind_act,period_start_utc,period_end_utc,date,year,month,day,dayofyear,hour,week,dayofweek
131420,01/10/2018 00:00:00 - 01/10/2018 00:15:00,NaN,NaN,2018-09-30 22:00:00,2018-09-30 22:15:00,2018-09-30,2018,9,30,273,22,39,6
131421,01/10/2018 00:15:00 - 01/10/2018 00:30:00,NaN,NaN,2018-09-30 22:15:00,2018-09-30 22:30:00,2018-09-30,2018,9,30,273,22,39,6
131422,01/10/2018 00:30:00 - 01/10/2018 00:45:00,NaN,NaN,2018-09-30 22:30:00,2018-09-30 22:45:00,2018-09-30,2018,9,30,273,22,39,6
131423,01/10/2018 00:45:00 - 01/10/2018 01:00:00,NaN,NaN,2018-09-30 22:45:00,2018-09-30 23:00:00,2018-09-30,2018,9,30,273,22,39,6
131424,01/10/2018 01:00:00 - 01/10/2018 01:15:00,NaN,NaN,2018-09-30 23:00:00,2018-09-30 23:15:00,2018-09-30,2018,9,30,273,23,39,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...
166455,30/09/2018 22:45:00 - 30/09/2018 23:00:00,NaN,NaN,2018-09-30 20:45:00,2018-09-30 21:00:00,2018-09-30,2018,9,30,273,20,39,6
166456,30/09/2018 23:00:00 - 30/09/2018 23:15:00,NaN,NaN,2018-09-30 21:00:00,2018-09-30 21:15:00,2018-09-30,2018,9,30,273,21,39,6
166457,30/09/2018 23:15:00 - 30/09/2018 23:30:00,NaN,NaN,2018-09-30 21:15:00,2018-09-30 21:30:00,2018-09-30,2018,9,30,273,21,39,6
166458,30/09/2018 23:30:00 - 30/09/2018 23:45:00,NaN,NaN,2018-09-30 21:30:00,2018-09-30 21:45:00,2018-09-30,2018,9,30,273,21,39,6


In [19]:
df_utc_op.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 420768 entries, 0 to 420767
Data columns (total 13 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   period            420768 non-null  object        
 1   off_wind_da       385632 non-null  float64       
 2   off_wind_act      385728 non-null  float64       
 3   period_start_utc  420768 non-null  datetime64[ns]
 4   period_end_utc    420768 non-null  datetime64[ns]
 5   date              420768 non-null  datetime64[ns]
 6   year              420768 non-null  int32         
 7   month             420768 non-null  int32         
 8   day               420768 non-null  int32         
 9   dayofyear         420768 non-null  int32         
 10  hour              420768 non-null  int32         
 11  week              420768 non-null  Int64         
 12  dayofweek         420768 non-null  int32         
dtypes: Int64(1), datetime64[ns](3), float64(2), int32(6), objec

In [20]:
group_cols = [
    "date", "year", "month", "day", "dayofyear", "hour", "week", "dayofweek"
]

df_utc_h = (
    df_utc_op
    .groupby(group_cols, as_index=False)
    .agg(
        off_wind_da=("off_wind_da", "mean"),
        off_wind_act=("off_wind_act", "mean"),
        period_start_utc=("period_start_utc", "min"),
        period_end_utc=("period_end_utc", "max"),
        c_by_hour=("year", "size"),   # count rows per group
    )
)
df_utc_h.head()

,date,year,month,day,dayofyear,hour,week,dayofweek,off_wind_da,off_wind_act,period_start_utc,period_end_utc,c_by_hour
0,2014-12-31,2014,12,31,365,23,1,2,258.50,521.1775,2014-12-31 23:00:00,2015-01-01 00:00:00,4
1,2015-01-01,2015,1,1,1,0,1,3,258.75,520.8100,2015-01-01 00:00:00,2015-01-01 01:00:00,4
2,2015-01-01,2015,1,1,1,1,1,3,258.50,519.0325,2015-01-01 01:00:00,2015-01-01 02:00:00,4
3,2015-01-01,2015,1,1,1,2,1,3,259.00,522.5550,2015-01-01 02:00:00,2015-01-01 03:00:00,4
4,2015-01-01,2015,1,1,1,3,1,3,260.50,524.6200,2015-01-01 03:00:00,2015-01-01 04:00:00,4


In [21]:
df_utc_h.tail()

,date,year,month,day,dayofyear,hour,week,dayofweek,off_wind_da,off_wind_act,period_start_utc,period_end_utc,c_by_hour
96427,2025-12-31,2025,12,31,365,18,1,2,5430.0475,5418.4900,2025-12-31 18:00:00,2025-12-31 19:00:00,4
96428,2025-12-31,2025,12,31,365,19,1,2,5494.9900,5015.0525,2025-12-31 19:00:00,2025-12-31 20:00:00,4
96429,2025-12-31,2025,12,31,365,20,1,2,5359.1000,4390.0575,2025-12-31 20:00:00,2025-12-31 21:00:00,4
96430,2025-12-31,2025,12,31,365,21,1,2,5389.0800,3363.9350,2025-12-31 21:00:00,2025-12-31 22:00:00,4
96431,2025-12-31,2025,12,31,365,22,1,2,5487.5425,2714.7750,2025-12-31 22:00:00,2025-12-31 23:00:00,4


In [22]:
df_utc_h["c_by_hour"].value_counts()

c_by_hour
4    87672
8     8760
Name: count, dtype: int64

In [23]:
group_cols = [
    "date", "year", "month", "day", "dayofyear", "week", "dayofweek"
]

df_utc_d = (
    df_utc_h
    .groupby(group_cols, as_index=False)
    .agg(
        off_wind_da=("off_wind_da", "mean"),
        off_wind_act=("off_wind_act", "mean"),
        period_start_utc=("period_start_utc", "min"),
        period_end_utc=("period_end_utc", "max"),
        c_by_day=("year", "size"),   # count rows per group
    )
)
df_utc_d.head()

,date,year,month,day,dayofyear,week,dayofweek,off_wind_da,off_wind_act,period_start_utc,period_end_utc,c_by_day
0,2014-12-31,2014,12,31,365,1,2,258.500000,521.177500,2014-12-31 23:00:00,2015-01-01,1
1,2015-01-01,2015,1,1,1,1,3,264.348958,516.931250,2015-01-01 00:00:00,2015-01-02,24
2,2015-01-02,2015,1,2,2,1,4,269.208333,443.582604,2015-01-02 00:00:00,2015-01-03,24
3,2015-01-03,2015,1,3,3,1,5,252.842391,510.003021,2015-01-03 00:00:00,2015-01-04,24
4,2015-01-04,2015,1,4,4,1,6,192.125000,483.414687,2015-01-04 00:00:00,2015-01-05,24


In [24]:
df_utc_d.tail()

,date,year,month,day,dayofyear,week,dayofweek,off_wind_da,off_wind_act,period_start_utc,period_end_utc,c_by_day
4014,2025-12-27,2025,12,27,361,52,5,4095.355417,4762.382500,2025-12-27,2025-12-28 00:00:00,24
4015,2025-12-28,2025,12,28,362,52,6,2255.130104,2541.367708,2025-12-28,2025-12-29 00:00:00,24
4016,2025-12-29,2025,12,29,363,1,0,5641.653854,6624.986354,2025-12-29,2025-12-30 00:00:00,24
4017,2025-12-30,2025,12,30,364,1,1,5544.454479,5786.144583,2025-12-30,2025-12-31 00:00:00,24
4018,2025-12-31,2025,12,31,365,1,2,5669.784130,5605.364783,2025-12-31,2025-12-31 23:00:00,23


In [25]:
df_utc_d["c_by_day"].value_counts()

c_by_day
24    4017
1        1
23       1
Name: count, dtype: int64

In [26]:
group_cols = [
    "year"
]

df_utc_y = (
    df_utc_d
    .groupby(group_cols, as_index=False)
    .agg(
        off_wind_da=("off_wind_da", "mean"),
        off_wind_act=("off_wind_act", "mean"),
        period_start_utc=("period_start_utc", "min"),
        period_end_utc=("period_end_utc", "max"),
        c_by_year=("year", "size"),   # count rows per group
    )
)
df_utc_y.head(20)

,year,off_wind_da,off_wind_act,period_start_utc,period_end_utc,c_by_year
0,2014,258.500000,521.177500,2014-12-31 23:00:00,2015-01-01 00:00:00,1
1,2015,692.575045,932.020181,2015-01-01 00:00:00,2016-01-01 00:00:00,365
2,2016,1364.492121,1376.834332,2016-01-01 00:00:00,2017-01-01 00:00:00,366
3,2017,1972.704539,1987.874390,2017-01-01 00:00:00,2018-01-01 00:00:00,365
4,2018,2193.005959,2176.779743,2018-01-01 00:00:00,2019-01-01 00:00:00,365
5,2019,2807.783071,2783.220802,2019-01-01 00:00:00,2020-01-01 00:00:00,365
6,2020,3096.313828,3060.444596,2020-01-01 00:00:00,2021-01-01 00:00:00,366
7,2021,2737.739251,2741.536458,2021-01-01 00:00:00,2022-01-01 00:00:00,365
8,2022,2837.045239,2824.967393,2022-01-01 00:00:00,2023-01-01 00:00:00,365
9,2023,2649.746606,2685.216202,2023-01-01 00:00:00,2024-01-01 00:00:00,365


In [27]:
# Check NaN only in one column ('price')
rows_with_missing_price = df_utc_op[df_utc_op["off_wind_da"].isna()]
rows_with_missing_price

,period,off_wind_da,off_wind_act,period_start_utc,period_end_utc,date,year,month,day,dayofyear,hour,week,dayofweek
288,04/01/2015 00:00:00 - 04/01/2015 00:15:00,NaN,518.58,2015-01-03 23:00:00,2015-01-03 23:15:00,2015-01-03,2015,1,3,3,23,1,5
289,04/01/2015 00:15:00 - 04/01/2015 00:30:00,NaN,518.95,2015-01-03 23:15:00,2015-01-03 23:30:00,2015-01-03,2015,1,3,3,23,1,5
290,04/01/2015 00:30:00 - 04/01/2015 00:45:00,NaN,519.27,2015-01-03 23:30:00,2015-01-03 23:45:00,2015-01-03,2015,1,3,3,23,1,5
291,04/01/2015 00:45:00 - 04/01/2015 01:00:00,NaN,518.06,2015-01-03 23:45:00,2015-01-04 00:00:00,2015-01-03,2015,1,3,3,23,1,5
292,04/01/2015 01:00:00 - 04/01/2015 01:15:00,NaN,518.62,2015-01-04 00:00:00,2015-01-04 00:15:00,2015-01-04,2015,1,4,4,0,1,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...
166455,30/09/2018 22:45:00 - 30/09/2018 23:00:00,NaN,NaN,2018-09-30 20:45:00,2018-09-30 21:00:00,2018-09-30,2018,9,30,273,20,39,6
166456,30/09/2018 23:00:00 - 30/09/2018 23:15:00,NaN,NaN,2018-09-30 21:00:00,2018-09-30 21:15:00,2018-09-30,2018,9,30,273,21,39,6
166457,30/09/2018 23:15:00 - 30/09/2018 23:30:00,NaN,NaN,2018-09-30 21:15:00,2018-09-30 21:30:00,2018-09-30,2018,9,30,273,21,39,6
166458,30/09/2018 23:30:00 - 30/09/2018 23:45:00,NaN,NaN,2018-09-30 21:30:00,2018-09-30 21:45:00,2018-09-30,2018,9,30,273,21,39,6


In [28]:
# Check NaN only in one column ('price')
rows_with_missing_price = df_utc_op[df_utc_op["off_wind_act"].isna()]
rows_with_missing_price

,period,off_wind_da,off_wind_act,period_start_utc,period_end_utc,date,year,month,day,dayofyear,hour,week,dayofweek
131420,01/10/2018 00:00:00 - 01/10/2018 00:15:00,NaN,NaN,2018-09-30 22:00:00,2018-09-30 22:15:00,2018-09-30,2018,9,30,273,22,39,6
131421,01/10/2018 00:15:00 - 01/10/2018 00:30:00,NaN,NaN,2018-09-30 22:15:00,2018-09-30 22:30:00,2018-09-30,2018,9,30,273,22,39,6
131422,01/10/2018 00:30:00 - 01/10/2018 00:45:00,NaN,NaN,2018-09-30 22:30:00,2018-09-30 22:45:00,2018-09-30,2018,9,30,273,22,39,6
131423,01/10/2018 00:45:00 - 01/10/2018 01:00:00,NaN,NaN,2018-09-30 22:45:00,2018-09-30 23:00:00,2018-09-30,2018,9,30,273,22,39,6
131424,01/10/2018 01:00:00 - 01/10/2018 01:15:00,NaN,NaN,2018-09-30 23:00:00,2018-09-30 23:15:00,2018-09-30,2018,9,30,273,23,39,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...
166455,30/09/2018 22:45:00 - 30/09/2018 23:00:00,NaN,NaN,2018-09-30 20:45:00,2018-09-30 21:00:00,2018-09-30,2018,9,30,273,20,39,6
166456,30/09/2018 23:00:00 - 30/09/2018 23:15:00,NaN,NaN,2018-09-30 21:00:00,2018-09-30 21:15:00,2018-09-30,2018,9,30,273,21,39,6
166457,30/09/2018 23:15:00 - 30/09/2018 23:30:00,NaN,NaN,2018-09-30 21:15:00,2018-09-30 21:30:00,2018-09-30,2018,9,30,273,21,39,6
166458,30/09/2018 23:30:00 - 30/09/2018 23:45:00,NaN,NaN,2018-09-30 21:30:00,2018-09-30 21:45:00,2018-09-30,2018,9,30,273,21,39,6


## saving our files by hour

In [29]:
df_utc_h.to_csv("../../data_cleaned/by_source/03_RES_WIND_OFFSHORE.csv", index=False)